In [27]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
# Disable truncation for pandas output
pd.set_option('display.max_colwidth', None)

In [29]:
import os
import pandas as pd

# Define the folder containing the dataset
folder_path = "/kaggle/input/include"

# Lists to store video paths and labels
video_files = []
labels = []

# Read the folder structure
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".MOV"):  # Check for video files
            video_path = os.path.join(root, file)
            video_files.append(video_path)

            # Extract the label from the folder name
            # Assuming the folder names are like "Adjectives_1of8"
            label = os.path.basename(root).replace('_', ' ')
#             print(label)
            labels.append(label.split(" ")[-1])

# Create a DataFrame
df = pd.DataFrame({"video_name": video_files, "tag": labels})

# Display the DataFrame
# print(df.head())


In [30]:
xdf = df[0:100]

In [33]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
xdf['tag'] = label_encoder.fit_transform(xdf['tag'])
xdf

/tmp/ipykernel_34/2762686943.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xdf['tag'] = label_encoder.fit_transform(xdf['tag'])


,video_name,tag
0,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5295.MOV,2
1,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5138.MOV,2
2,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5297.MOV,2
3,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5136.MOV,2
4,/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_9408.MOV,2
...,...,...
95,/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_9250.MOV,0
96,/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_9333.MOV,0
97,/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_9332.MOV,0
98,/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_5139.MOV,0


In [34]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.15, random_state=42)

In [8]:
!pip install -q pytorchvideo transformers[sentencepiece] evaluate
!pip install accelerate -U
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [35]:
import av

import pandas as pd
import os
import glob
import io
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score
from transformers import AutoImageProcessor, VideoMAEModel,  AdamW
import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from sklearn.metrics import f1_score, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [36]:
train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

In [37]:
def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.array(np.stack([x.to_ndarray(format="rgb24") for x in frames]))

In [38]:
def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


In [39]:
def get_image_processor_inputs(file_path, image_processor):
    container = av.open(file_path)
    indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
    video = read_video_pyav(container, indices)
    inputs = image_processor(list(video), return_tensors="pt")
    
#     video_tensor = inputs['pixel_values']
#     if video_tensor.dim() == 4:  # If the shape is (num_frames, num_channels, height, width)
#         video_tensor = video_tensor.unsqueeze(0) 
    return inputs
#     return video_tensor

In [40]:
class VideoClassificationDataset(Dataset):
    def __init__(self, root_dir, data_df, image_processor):
        self.data = data_df.dropna()
        self.root_dir = root_dir
        self.image_processor = image_processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        video_path = self.data.iloc[index]['video_name']

#         video_path = os.path.join(self.root_dir, image_path)

        video_encodings = get_image_processor_inputs(video_path, image_processor).pixel_values
#         video_encodings = get_image_processor_inputs(video_path, image_processor)
        labels = np.array(self.data.iloc[index]["tag"])

        return video_encodings, labels
#         return  {"pixel_values": video_encodings, "labels": torch.tensor(labels)}

In [41]:

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
video_encoder = VideoMAEModel.from_pretrained("MCG-NJU/videomae-base")
# video_encoder = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base", num_labels=train_df['tag'].nunique())

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [47]:
train_dataset = VideoClassificationDataset("/kaggle/input/include", train_df,  image_processor)
test_dataset = VideoClassificationDataset("/kaggle/input/include",  test_df, image_processor)
val_dataset = VideoClassificationDataset("/kaggle/input/include",  val_df, image_processor)

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [18]:
# for i in train_loader:
#     print(i[0])
#     print(type(i))
#     break

In [43]:
class VideoClassifier(nn.Module):
    def __init__(self, num_labels, video_encoder):
        super(VideoClassifier, self).__init__()
        self.video_encoder = video_encoder
        self.classifier = nn.Sequential(
            nn.Linear(self.video_encoder.config.hidden_size, 2048),
            nn.ReLU(), 
            
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, num_labels)
        )

    def forward(self, video_features):
        outputs = self.video_encoder(video_features)
        pooled_output = outputs['last_hidden_state'][:, 0, :]
        logits = self.classifier(pooled_output)
        return logits


In [44]:
num_labels = train_df['tag'].nunique()

model = VideoClassifier(num_labels, video_encoder).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
criterion = nn.CrossEntropyLoss()


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [48]:

# Define the training function
def train(model, train_loader, val_loader, optimizer,  criterion, device, num_epochs):

    best_accuracy = 0.0

    for epoch in range(num_epochs):

        model.train()

        for i, batch in enumerate(train_loader):

            video_features, labels = batch

            if video_features.shape[0] == 1:
              video_features = video_features.squeeze(0)
            else:
              video_features = video_features.squeeze()

            video_features = video_features.to(device)

            labels = labels.view(-1)
            labels = labels.to(device)

            optimizer.zero_grad()

            logits = model(video_features)

            loss = criterion(logits, labels)
            loss.backward()

            optimizer.step()

            if (i+1) % 8 == 0:
                print(f'Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Train Loss: {loss.item() :.4f}')
                train_loss = 0.0

        val_loss, val_accuracy, val_f1, _ , _ = evaluate(model, val_loader, device)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_model.pt')

        print("========================================================================================")
        print(f'Epoch {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}, best_accuracy: {best_accuracy:.4f}')
        print("========================================================================================")



In [49]:

def evaluate(model, data_loader,  device):

    print("evaluate started")
    all_labels = []
    all_preds = []
    total_loss = 0.0

    with torch.no_grad():

        for i, batch in enumerate(data_loader):

          video_features, labels = batch

          if video_features.shape[0] == 1:

            video_features = video_features.squeeze(0)
          else:
            video_features = video_features.squeeze()

          video_features = video_features.to(device)

          labels = labels.view(-1)
          labels = labels.to(device)

          logits = model(video_features)
          loss = criterion(logits, labels)
          total_loss += loss.item()

          _, preds = torch.max(logits, 1)

          all_labels.append(labels.cpu().numpy())
          all_preds.append(preds.cpu().numpy())


    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)

    loss = total_loss / len(data_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    return loss, accuracy, f1, all_labels, all_preds



In [25]:
# torch.cuda.reset_max_memory_allocated()

In [50]:
torch.cuda.empty_cache()
video_encoder.gradient_checkpointing_enable()
num_epochs = 5
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs)

Epoch 1/5, Batch 8/196, Train Loss: 5.5798
Epoch 1/5, Batch 16/196, Train Loss: 5.5555
Epoch 1/5, Batch 24/196, Train Loss: 5.5621
Epoch 1/5, Batch 32/196, Train Loss: 5.5607
Epoch 1/5, Batch 40/196, Train Loss: 5.5480
Epoch 1/5, Batch 48/196, Train Loss: 5.5605
Epoch 1/5, Batch 56/196, Train Loss: 5.5402
Epoch 1/5, Batch 64/196, Train Loss: 5.5424
Epoch 1/5, Batch 72/196, Train Loss: 5.5427
Epoch 1/5, Batch 80/196, Train Loss: 5.5505
Epoch 1/5, Batch 88/196, Train Loss: 5.5521
Epoch 1/5, Batch 96/196, Train Loss: 5.5387
Epoch 1/5, Batch 104/196, Train Loss: 5.5437
Epoch 1/5, Batch 112/196, Train Loss: 5.5295
Epoch 1/5, Batch 120/196, Train Loss: 5.5185
Epoch 1/5, Batch 128/196, Train Loss: 5.5356
Epoch 1/5, Batch 136/196, Train Loss: 5.5527
Epoch 1/5, Batch 144/196, Train Loss: 5.4868
Epoch 1/5, Batch 152/196, Train Loss: 5.4777
Epoch 1/5, Batch 160/196, Train Loss: 5.5161
Epoch 1/5, Batch 168/196, Train Loss: 5.4501
Epoch 1/5, Batch 176/196, Train Loss: 5.4738
Epoch 1/5, Batch 184/19

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch 1/5, Val Loss: 5.4357, Val Accuracy: 0.0254, Val F1: 0.0014, best_accuracy: 0.0254
Epoch 2/5, Batch 8/196, Train Loss: 5.3618



KeyboardInterrupt

